In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle


In [26]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [27]:
data = data.drop(columns=['RowNumber','CustomerId','Surname'])
data.head()


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [28]:
label_encoder_regression = LabelEncoder()
data['Gender'] = label_encoder_regression.fit_transform(data['Gender'])

data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [29]:
one_hot_regression = OneHotEncoder(drop='first')
geo_encoder = one_hot_regression.fit_transform(data[['Geography']]).toarray()
geo_encoder_df = pd.DataFrame(geo_encoder, columns = one_hot_regression.get_feature_names_out())

data = pd.concat([data.drop('Geography', axis=1), geo_encoder_df],axis=1)

data.head()


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,1.0


In [30]:
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [31]:
with open('label_encoder_regression.pkl','wb') as f:
    pickle.dump(label_encoder_regression,f)

with open('one_hot_regression.pkl','wb') as f:
    pickle.dump(one_hot_regression,f)

with open('scaler.pkl','wb') as f:
    pickle.dump(scaler,f)



In [32]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [33]:
model = Sequential([
    Dense(64,activation='relu', input_shape= (X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1,activation='linear')

])

c:\Users\mkuma\OneDrive\Desktop\ANN_FIRST_PROJECT\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [34]:
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,881 (11.25 KB)

 Trainable params: 2,881 (11.25 KB)

 Non-trainable params: 0 (0.00 B)

In [35]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.MeanAbsoluteError()

In [36]:
model.compile(optimizer=opt,loss=loss,metrics= ['mae'])

In [37]:
import datetime


In [38]:
log_dir = "log/regression/" + datetime.datetime.now().strftime('%Y%M%d-%H%M%S')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1)


In [39]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=100,restore_best_weights=True)

In [40]:
history = model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorboard_callback,early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 99743.1797 - mae: 99743.1797 - val_loss: 98530.0938 - val_mae: 98530.0938
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 100712.7344 - mae: 100712.7344 - val_loss: 97187.2891 - val_mae: 97187.2891
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 98200.7891 - mae: 98200.7891 - val_loss: 93779.4453 - val_mae: 93779.4453
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 94450.4766 - mae: 94450.4766 - val_loss: 88055.9141 - val_mae: 88055.9141
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 87833.7969 - mae: 87833.7969 - val_loss: 80492.3828 - val_mae: 80492.3828
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 79959.7891 - mae: 79959.7891 - val_loss: 72139.5469 - val_mae: 72139.5469
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 71134.4609 - mae: 71134.4609 - val_loss: 64510.9336 - val_mae: 64510.9336
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step 

In [41]:
model.evaluate(X_test,y_test)


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 50921.1992 - mae: 50921.1992 


[50241.1875, 50241.1875]

In [42]:
model.save('regression_model.h5')

In [43]:
%load_ext tensorboard
%tensorboard --logdir logs/regression/


Reusing TensorBoard on port 6006 (pid 5584), started 15:04:23 ago. (Use '!kill 5584' to kill it.)